In [ ]:
import random
from datetime import datetime, timedelta
import pandas as pd
from tabulate import tabulate

def generate_random_times(num_clients, max_arrival_time, max_service_time):
    """

    Generate random arrival times and service times for clients.

    Args:
        num_clients (int): Number of clients to simulate.
        max_arrival_time (int): Maximum time between arrivals.
        max_service_time (int): Maximum service time for each client.

    Returns:
        tuple: random arrival time and service times

    """
    arrival_times = [random.randint(1, max_arrival_time) for _ in range(num_clients)]
    service_times = [random.randint(1, max_service_time) for _ in range(num_clients)]
    return arrival_times, service_times

def simulate_atm(num_clients, max_arrival_time, max_service_time):
    """
    Simulate the operation of an ATM with random arrival and service times.

    Args:
        num_clients (int): Number of clients to simulate.
        max_arrival_time (int): Maximum time between arrivals.
        max_service_time (int): Maximum service time for each client.

    Returns:
        tuple: DataFrame containing simulation results and performance metrics.
    """
    arrival_times, service_times = generate_random_times(num_clients, max_arrival_time, max_service_time)

    data = []
    current_time = datetime.now()
    last_service_end_time = current_time

    total_wait_time = 0
    total_idle_time = 0
    total_service_time = 0
    waiting_clients = 0

    for i in range(num_clients):
        # Calculate arrival and service times for each client
        arrival_time = current_time + timedelta(minutes=sum(arrival_times[:i+1]))
        service_time = service_times[i]

        if i == 0:
            start_service_time = arrival_time
            wait_time = 0
            idle_time = 0
        else:
            if arrival_time > last_service_end_time:
                # If there is idle time between clients
                idle_time = (arrival_time - last_service_end_time).total_seconds() / 60.0
                total_idle_time += idle_time
                start_service_time = arrival_time
                wait_time = 0
            else:
                # If the client has to wait in line
                wait_time = (last_service_end_time - arrival_time).total_seconds() / 60.0
                total_wait_time += wait_time
                start_service_time = last_service_end_time
                idle_time = 0
                waiting_clients += 1

        end_service_time = start_service_time + timedelta(minutes=service_time)
        service_duration = service_time

        # Store data for analysis
        data.append([
            i + 1,
            arrival_times[i],
            arrival_time.strftime("%H:%M:%S"),
            service_time,
            start_service_time.strftime("%H:%M:%S"),
            end_service_time.strftime("%H:%M:%S"),
            wait_time,
            idle_time
        ])

        last_service_end_time = end_service_time
        total_service_time += service_duration

    # Create DataFrame for analysis
    df = pd.DataFrame(data, columns=[
        'Cliente', 'Tiempo entre llegadas (minutos)', 'Hora de llegada', 'Tiempo de trámite (minutos)',
        'Inicio del servicio', 'Término del servicio', 'Tiempo de espera (minutos)', 'Tiempo de inactividad del ATM (minutos)'
    ])

    # Calculate performance metrics
    average_wait_time = total_wait_time / num_clients
    wait_probability = waiting_clients / num_clients
    idle_percentage = (total_idle_time / (total_idle_time + total_service_time)) * 100
    average_service_time = total_service_time / num_clients

    return df, average_wait_time, wait_probability, idle_percentage, average_service_time

# Parameters
num_clients = 10
max_arrival_time = 10
max_service_time = 15

# Simulate ATM operation and print results
df, avg_wait_time, wait_prob, idle_pct, avg_service_time = simulate_atm(num_clients, max_arrival_time, max_service_time)

# Printing statements
print(tabulate(df, headers='keys', tablefmt='grid'))
print(f"\nTiempo de espera promedio por cliente: {avg_wait_time:.2f} minutos")
print(f"Probabilidad de que un cliente espere en la fila: {wait_prob*100} %")
print(f"Porcentaje de tiempo que el cajero estuvo inactivo: {idle_pct:.2f}%")
print(f"Tiempo promedio de servicio: {avg_service_time:.2f} minutos")

# Write output to a file
output = []

output.append(tabulate(df, headers='keys', tablefmt='grid'))
output.append(f"\nTiempo de espera promedio por cliente: {avg_wait_time:.2f} minutos")
output.append(f"Probabilidad de que un cliente espere en la fila: {wait_prob*100} %")
output.append(f"Porcentaje de tiempo que el cajero estuvo inactivo: {idle_pct:.2f}%")
output.append(f"Tiempo promedio de servicio: {avg_service_time:.2f} minutos")

with open('output.txt', 'w') as file:
    for line in output:
        file.write(line + '\n')


+----+-----------+-----------------------------------+-------------------+-------------------------------+-----------------------+------------------------+------------------------------+-------------------------------------------+
|    |   Cliente |   Tiempo entre llegadas (minutos) | Hora de llegada   |   Tiempo de trámite (minutos) | Inicio del servicio   | Término del servicio   |   Tiempo de espera (minutos) |   Tiempo de inactividad del ATM (minutos) |
+====+===========+===================================+===================+===============================+=======================+========================+==============================+===========================================+
|  0 |         1 |                                 1 | 22:49:15          |                             7 | 22:49:15              | 22:56:15               |                            0 |                                         0 |
+----+-----------+-----------------------------------+-------------------+--